# Storing to a Database

This example notebook walks through an example of storing Market Data in a `sqlite` database.

- Backfilling
- Updating
- Reference Data
- Example Queries

In [1]:
import spgci as ci
import sqlite3
import pandas as pd
from datetime import date

In [2]:
mdd = ci.MarketData()

## Backfill data
Starting out we will want to populate our database with all historical records

For the sake of this example, we assume today is `2023-01-01`, and the symbols we are interested in are `PCAAS00`, `AAGZU00` and `PCAAQ00`

In [3]:
todays_date = date(2023,1,1) # only neceessary for this example.
symbols = ['PCAAS00', 'AAGZU00', 'PCAAQ00']

Fetch data for the symbols we are interested in. The date filter is only included in this example to show how to do updates. In a real scenario this can be omitted.

In [4]:
df = mdd.get_assessments_by_symbol_historical(symbol=symbols, 
                                              assess_date_lte=todays_date, # only necessary for this example
                                              paginate=True)
df

Fetching...: 100%|██████████| 6/6 [00:02<00:00,  2.19it/s]


,bate,value,assessDate,isCorrected,modDate,symbol
0,h,26.650,2001-06-20,Y,2001-06-20 21:39:33,AAGZU00
1,l,26.560,2001-06-20,Y,2001-06-20 21:39:33,AAGZU00
2,h,27.100,2001-06-21,Y,2001-06-21 21:18:09,AAGZU00
3,l,26.990,2001-06-21,Y,2001-06-21 21:18:09,AAGZU00
4,h,27.650,2001-06-22,Y,2001-06-22 21:12:30,AAGZU00
...,...,...,...,...,...,...
56400,h,80.690,2022-12-29,N,2022-12-29 19:13:14,PCAAS00
56401,l,80.680,2022-12-29,N,2022-12-29 19:13:14,PCAAS00
56402,c,81.325,2022-12-30,N,2022-12-30 14:13:00,PCAAS00
56403,h,81.330,2022-12-30,N,2022-12-30 14:13:00,PCAAS00


### Database setup
- setup a connection to our DB
- setup the schema for our assessments table - `spgci_assessments`
- generate a prepared statement

In [5]:
conn = sqlite3.connect('db.db', detect_types=sqlite3.PARSE_COLNAMES|sqlite3.PARSE_DECLTYPES)
conn.execute(pd.io.sql.get_schema(df, 'spgci_assessments', con=conn, keys=['symbol', 'bate', 'assessDate']));

In [6]:
def prepare_stmt(table, columns):
  cols = f"({','.join(columns)})"
  values = f"({','.join(['?' for i in columns])})"
  stmt = f"INSERT OR REPLACE INTO {table} {cols} VALUES {values}"

  return stmt

stmt = prepare_stmt("spgci_assessments", df.columns)
stmt

'INSERT OR REPLACE INTO spgci_assessments (bate,value,assessDate,isCorrected,modDate,symbol) VALUES (?,?,?,?,?,?)'

### Insert records
- insert function
- calling insert function per row in `df`
- commiting the transaction

In [7]:
def insert(conn, stmt, row):
  values = []
  for r in row:
    if isinstance(r, pd.Timestamp):
      values.append(r.to_pydatetime())
    elif isinstance(r, list):
      values.append(str(r))
    else:
      values.append(r)
      
  conn.execute(stmt,values) 


In [9]:
df.apply(lambda x: insert(conn, stmt, x), axis=1)
conn.commit()

### Selecting Data




In [10]:
r = conn.execute("""
    select * from spgci_assessments 
    order by assessDate desc
    limit 10;
""")

for row in r:
    print(row)

('c', 76.125, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13, 14), 'AAGZU00')
('h', 76.14, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13, 14), 'AAGZU00')
('l', 76.11, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13, 14), 'AAGZU00')
('c', 82.64, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 12, 56, 26), 'PCAAQ00')
('h', 82.65, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 12, 56, 26), 'PCAAQ00')
('l', 82.63, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 12, 56, 26), 'PCAAQ00')
('c', 81.325, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13), 'PCAAS00')
('h', 81.33, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13), 'PCAAS00')
('l', 81.32, datetime.datetime(2022, 12, 30, 0, 0), 'N', datetime.datetime(2022, 12, 30, 14, 13), 'PCA

# Fetch Deltas
After backfilling is complete, its time to move on to grabbing updates.

In this example it is done just once, but you would want to schedule this daily (or hourly) to add more data as it becomes avaialble. Shifting the `date` filter forward each time.


In [11]:
df_since = mdd.get_assessments_by_symbol_historical(symbol=symbols, assess_date_gte=todays_date, paginate=True)
df_since.head()

,bate,value,assessDate,isCorrected,modDate,symbol
0,c,76.090,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
1,h,76.110,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
2,l,76.070,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
3,c,70.445,2023-01-04,N,2023-01-04 18:50:34,AAGZU00
4,h,70.460,2023-01-04,N,2023-01-04 18:50:34,AAGZU00


same steps as before:
- prepare statement
- insert rows
- commit update

In [12]:
stmt = prepare_stmt("spgci_assessments", df_since.columns)
df_since.apply(lambda x: insert(conn, stmt, x), axis=1)
conn.commit()

validate data has been added

In [13]:
r = conn.execute("""
    select * from spgci_assessments 
    order by assessDate desc
    limit 10;
""")
for row in r:
    print(row)

('c', 73.965, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 2, 7), 'AAGZU00')
('h', 73.98, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 2, 7), 'AAGZU00')
('l', 73.95, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 2, 7), 'AAGZU00')
('c', 78.8, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 15, 57, 57), 'PCAAQ00')
('h', 78.81, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 15, 57, 57), 'PCAAQ00')
('l', 78.79, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 15, 57, 57), 'PCAAQ00')
('c', 77.84, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 1, 48), 'PCAAS00')
('h', 77.85, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 1, 48), 'PCAAS00')
('l', 77.83, datetime.datetime(2023, 3, 28, 0, 0), 'N', datetime.datetime(2023, 3, 28, 17, 1, 48), 'PCAAS00')
('c', 71.4

## Reference Data
Now, lets add reference data so we know the description, currency, uom, etc..

In [14]:
ref = mdd.get_symbols(mdc="RI", paginate=True)
ref

,symbol,description,assessment_frequency,bate_code,benchmark,holiday_schedule,commodity,commodity_grade,contract_type,currency,...,kinematic_viscosity,pour_point,flash_point,originating_region,originating_region_basis,curve_code,curve_name,cargo_size,delivery_load,exchange
0,AJSVB00,Johan Sverdrup FOB North Sea vs North Sea Dtd ...,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Johan Sverdrup,Spot,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAGZU00,CPC Blend CIF Augusta,Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - CPC Blend,Spot,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AALIN00,Urals Recombined (RCMB),Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Ural,Spot,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PCAAQ00,Brent Mo01 (NextGen MOC),Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Brent,Forward,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AWTID00,WTI Midland DAP Basis Rotterdam vs Fwd Dated B...,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - WTI Midland,Spot,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,AAUFI03,BTC Dated Brent Strip MAvg,Monthly,"[c, h, l]",NaN,#Not Applicable,Crude oil,Platts - Crude Oil - Azeri Light,Strip,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,AALDG00,Med Dated Brent Strip 13-28 Day MAvg,Monthly,"[c, h, l]",NaN,#Not Applicable,Crude oil,Platts - Crude Oil - Brent,Strip,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
601,AAWXY03,Brent NX EFP Mo03 MAvg,Monthly,"[c, h, l]",NaN,#Not Applicable,Crude oil,Platts - Crude Oil - Brent,EFP,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
602,AAGXO00,Qua Iboe FOB Nigeria vs WAF Dtd Strip MAvg,Monthly,"[c, h, l]",NaN,#Not Applicable,Crude oil,Platts - Crude Oil - Qua Iboe,Spot,USD,...,3.92,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


same as before:
- create table
- prepare stmt
- insert rows
- commit

In [15]:
conn.execute(pd.io.sql.get_schema(ref, 'spgci_symbols', con=conn, keys=['symbol']));

In [16]:
stmt = prepare_stmt("spgci_symbols", ref.columns)
ref.apply(lambda x: insert(conn, stmt, x), axis=1)
conn.commit()

validate data has been inserted

In [17]:
r = conn.execute("""
    select * from spgci_symbols
    limit 3;
""")
for row in r:
    print(row)

('AJSVB00', 'Johan Sverdrup FOB North Sea vs North Sea Dtd Strip', 'Daily (weekday)', "['c']", 'Benchmark', 'CALLN00', 'Crude oil', 'Platts - Crude Oil - Johan Sverdrup', 'Spot', 'USD', 3.0, 'DW', "['MI Crude', 'Crudes: at 16:30 London']", "['CRU', 'RI']", 'Daily (weekdays)', 'Differential', 'North Sea', 'North Sea', 'Physical', 'FOB', 28.0, 600000.0, 'BBL', 'Active - Public', 0.8, '16:30 UK', 'BBL', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None)
('AAGZU00', 'CPC Blend CIF Augusta', 'Daily (weekday)', "['c', 'h', 'l']", 'Benchmark', 'CALLN00', 'Crude oil', 'Platts - Crude Oil - CPC Blend', 'Spot', 'USD', 3.0, 'DW', "['MI Crude', 'Crudes: at 16:30 London']", "['CRU', 'RI']", 'Daily (weekdays)', 'Flat', 'Mediterranean', 'Augusta', 'Physical', 'CIF', 43.5, None, None, 'Active - Public', 0.55, '16:30 UK', 'BBL', 'MED', 806.7, None, None, None, None, None, None, None, None, None, None, None, None, None)
('AALIN00', 'Urals Recombined (RCMB)', 'Daily

## Example Queries

- joining `spgci_assessments` with `spgci_symbols`

In [18]:
r = conn.execute("""
  SELECT
    s.symbol,
    s.description,
    s.currency,
    s.uom,
    a.bate,
    a.value,
    a.assessDate
  FROM
    spgci_symbols s
    JOIN spgci_assessments a ON s.symbol = a.symbol
  WHERE
    a.bate = "c"
  ORDER BY
    a.assessDate ASC
  LIMIT 10;
""")
for row in r:
    print(row)               

('PCAAQ00', 'Brent Mo01 (NextGen MOC)', 'USD', 'BBL', 'c', 107.62, datetime.datetime(2011, 8, 5, 0, 0))
('PCAAS00', 'Dated Brent', 'USD', 'BBL', 'c', 106.87, datetime.datetime(2011, 8, 5, 0, 0))
('AAGZU00', 'CPC Blend CIF Augusta', 'USD', 'BBL', 'c', 106.505, datetime.datetime(2011, 11, 1, 0, 0))
('PCAAQ00', 'Brent Mo01 (NextGen MOC)', 'USD', 'BBL', 'c', 105.96, datetime.datetime(2011, 11, 1, 0, 0))
('PCAAS00', 'Dated Brent', 'USD', 'BBL', 'c', 106.66, datetime.datetime(2011, 11, 1, 0, 0))
('AAGZU00', 'CPC Blend CIF Augusta', 'USD', 'BBL', 'c', 110.505, datetime.datetime(2011, 11, 2, 0, 0))
('PCAAQ00', 'Brent Mo01 (NextGen MOC)', 'USD', 'BBL', 'c', 109.7, datetime.datetime(2011, 11, 2, 0, 0))
('PCAAS00', 'Dated Brent', 'USD', 'BBL', 'c', 110.835, datetime.datetime(2011, 11, 2, 0, 0))
('AAGZU00', 'CPC Blend CIF Augusta', 'USD', 'BBL', 'c', 110.435, datetime.datetime(2011, 11, 3, 0, 0))
('PCAAQ00', 'Brent Mo01 (NextGen MOC)', 'USD', 'BBL', 'c', 109.64, datetime.datetime(2011, 11, 3, 0, 0